# Previsão de Estoque para 28 dias

In [ ]:
import pandas as pd
import numpy as np
from pmdarima import auto_arima
from statsmodels.tsa.stattools import adfuller
from google.cloud import bigquery
from google.oauth2 import service_account
import logging
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report

# Configurar logging com nível DEBUG para maior detalhamento
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

# Caminho para o arquivo de chave da conta de serviço
service_account_path = 'tfm-sa.json'
credentials = service_account.Credentials.from_service_account_file(service_account_path)
project_id = 'perseverance-332400'
dataset_id = 'TFM'

# Definir IDs de tabela separadas
data_table_id = 'ds_market'               # Tabela com dados originais
forecast_table_id = 'ds_market_forecast'  # Nova tabela para previsões

full_data_table_id = f'{project_id}.{dataset_id}.{data_table_id}'

# Função para configurar o cliente BigQuery usando credenciais específicas
def initialize_bigquery_client():
    return bigquery.Client(project=project_id, credentials=credentials)

# Função para extrair dados históricos do BigQuery
def get_historical_data_from_bigquery(query: str, client) -> pd.DataFrame:
    try:
        query_job = client.query(query)
        data = query_job.to_dataframe()
        logging.info(f"Dados extraídos do BigQuery com {data.shape[0]} linhas e {data.shape[1]} colunas.")
        logging.info(f"Colunas disponíveis nos dados extraídos: {data.columns.tolist()}")
        return data
    except Exception as e:
        logging.error(f"Erro ao extrair dados do BigQuery: {e}")
        return pd.DataFrame()  # Retorna um DataFrame vazio para evitar falhas subsequentes

# Função para preparar os dados
def prepare_sales_data(raw_data: pd.DataFrame, apply_log=False, aggregate_weekly=False) -> pd.DataFrame:
    raw_data = raw_data.copy()
    raw_data['date'] = pd.to_datetime(raw_data['date']).dt.tz_localize(None)  # Remove timezone
    logging.debug(f"Timezones após remoção: {raw_data['date'].dt.tz}")  # Deve ser None
    raw_data['sales'] = raw_data['sales'].fillna(0)
    
    if aggregate_weekly:
        # Agregação semanal
        sales_data = raw_data.groupby('date').agg({'sales': 'sum'}).resample('W').sum().sort_index()
        logging.info("Dados agregados semanalmente.")
    else:
        # Mantém dados diários
        sales_data = raw_data.groupby('date').agg({'sales': 'sum'}).sort_index()
    
    # Preencher lacunas diárias
    sales_data = sales_data.asfreq('D').fillna(0)
    logging.info("Dados de vendas preparados e limpos.")
    
    if apply_log:
        # Aplicar transformação logarítmica
        sales_data['sales_log'] = np.log1p(sales_data['sales'])  # log(1 + x)
        logging.debug("Transformação logarítmica aplicada.")
    
    return sales_data

# Função para detectar outliers usando o IQR
def detect_outliers(data: pd.DataFrame, column='sales', threshold=1.5):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers

# Função para visualizar a série temporal
def visualize_series(data: pd.DataFrame, store: str, item: str, save=False):
    plt.figure(figsize=(10, 5))
    plt.plot(data.index, data['sales'], label='Vendas')
    plt.title(f'Série Temporal - Loja {store} Item {item}')
    plt.xlabel('Data')
    plt.ylabel('Vendas')
    plt.legend()
    if save:
        plt.savefig(f'serie_temporal_{store}_{item}.png')
    else:
        plt.show()
    
    # Plotar resíduos se as colunas existirem
    if 'actual_sales' in data.columns and 'forecast_sales' in data.columns:
        plt.figure(figsize=(10, 5))
        residuals = data['actual_sales'] - data['forecast_sales']
        plt.plot(data.index, residuals, label='Resíduos')
        plt.hlines(0, data.index.min(), data.index.max(), colors='r', linestyles='dashed')
        plt.legend()
        plt.title(f'Residuals - Loja {store} Item {item}')
        plt.xlabel('Data')
        plt.ylabel('Resíduo')
        if save:
            plt.savefig(f'residuals_{store}_{item}.png')
        else:
            plt.show()

# Função para filtrar itens com muitas vendas zero
def filter_zero_sales(data: pd.DataFrame, threshold=0.9) -> bool:
    zero_sales_ratio = (data['sales'] == 0).mean()
    logging.debug(f"Proporção de vendas zero: {zero_sales_ratio*100:.2f}%")
    if zero_sales_ratio > threshold:
        logging.warning(f"Item com {zero_sales_ratio*100:.2f}% de vendas zero. Considerar remoção ou tratamento especial.")
        return False
    return True

# Função para ajustar o modelo usando auto_arima
def fit_auto_arima_model(train_series: pd.Series):
    # Verificar estacionariedade
    result = adfuller(train_series)
    logging.info(f"ADF Statistic: {result[0]}, p-value: {result[1]}")
    d = 0
    if result[1] > 0.05:
        logging.info("Série não estacionária. Aplicando diferenciação.")
        d = 1
        train_series = train_series.diff().dropna()
    else:
        logging.info("Série já é estacionária.")
    
    model = auto_arima(
        train_series,
        d=d,
        seasonal=False,
        trace=False,
        error_action='ignore',
        suppress_warnings=True,
        stepwise=True
    )
    logging.info(f"Melhor modelo ARIMA: ordem {model.order} com AIC {model.aic()}")
    return model

# Função para ajustar o modelo Prophet
def fit_prophet_model(train_series: pd.Series):
    train_df = train_series.reset_index().rename(columns={'date': 'ds', 'sales': 'y'})  # Usando 'sales' diretamente
    train_df['ds'] = train_df['ds'].dt.tz_localize(None)  # Remove timezone
    logging.debug(f"Timezones na coluna ds: {train_df['ds'].dt.tz}")  # Deve ser None
    model = Prophet(
        daily_seasonality=True,
        weekly_seasonality=True,
        yearly_seasonality=True  # Força a inclusão da sazonalidade anual
    )
    model.fit(train_df)
    logging.info("Modelo Prophet ajustado com sucesso.")
    return model

# Função para prever com Prophet
def predict_prophet_model(model, periods: int):
    future = model.make_future_dataframe(periods=periods, freq='D')  # Especificar frequência diária
    forecast = model.predict(future)
    yhat = forecast['yhat'][-periods:].values
    logging.debug(f"Previsões geradas: {len(yhat)} para {periods} períodos.")
    return yhat

# Função para validação cruzada em séries temporais com ARIMA
def cross_validate_arima(data: pd.Series, model_func, splits=5):
    tscv = TimeSeriesSplit(n_splits=splits)
    errors = []
    
    for fold, (train_index, test_index) in enumerate(tscv.split(data)):
        train, test = data.iloc[train_index], data.iloc[test_index]
        logging.debug(f"Fold {fold+1}: Treino {len(train)} dias, Teste {len(test)} dias")
        # Ajustar o modelo no conjunto de treinamento
        model = model_func(train)
        # Prever no conjunto de teste
        forecast = model.predict(n_periods=len(test))
        # Se a série foi diferenciada, acumular as previsões
        if model.order[1] > 0:
            forecast = np.cumsum(forecast) + train.iloc[-1]
        # Limitar previsões a valores não negativos
        forecast = np.maximum(forecast, 0)
        actual = test.values
        # Calcular o RMSE
        rmse = mean_squared_error(actual, forecast, squared=False)
        logging.debug(f"Fold {fold+1}: RMSE = {rmse}")
        errors.append(rmse)
    
    avg_rmse = np.mean(errors)
    logging.info(f"RMSE médio com validação cruzada: {avg_rmse}")
    return avg_rmse

# Função para calcular as métricas de erro
def calculate_metrics(actual, forecast):
    rmse = np.sqrt(mean_squared_error(actual, forecast))  # Calcula RMSE explicitamente
    mae = mean_absolute_error(actual, forecast)
    # Evitar divisão por zero no MAPE
    mask = actual != 0
    if not np.any(mask):
        mape = np.nan
        logging.warning("Todos os valores reais são zero. MAPE não pode ser calculado.")
    else:
        mape = mean_absolute_percentage_error(actual[mask], forecast[mask]) * 100  # MAPE em porcentagem
    
    logging.info(f"RMSE: {rmse}, MAE: {mae}, MAPE: {mape}%")
    return rmse, mae, mape

# Função para realizar validação cruzada com Prophet
def perform_prophet_cross_validation(model: Prophet, initial: str, period: str, horizon: str):
    df_cv = cross_validation(model, initial=initial, period=period, horizon=horizon, parallel="processes")
    df_p = performance_metrics(df_cv)
    logging.info(f"Métricas de validação cruzada:\n{df_p}")
    return df_p

# Função para carregar previsões no BigQuery
def store_forecast_results(data: pd.DataFrame, table_id: str, client):
    logging.info(f"Carregando previsões para a tabela {table_id} no BigQuery...")
    
    # Remover coluna 'actual_sales' se ela não for necessária na tabela de previsões
    if 'actual_sales' in data.columns:
        data = data.drop(columns=['actual_sales'])
    
    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_APPEND",
        schema_update_options=[bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION]
    )  # Evita sobrescrever a tabela original
    
    try:
        job = client.load_table_from_dataframe(data, table_id, job_config=job_config)
        job.result()
        logging.info("Previsões carregadas com sucesso no BigQuery.")
    except Exception as e:
        logging.error(f"Erro ao carregar previsões no BigQuery: {e}")

# Função para plotar previsões vs. dados reais
def plot_forecast_with_actual(train_data: pd.DataFrame, test_data: pd.DataFrame, forecast: pd.DataFrame, store: str, item: str, save=False):
    plt.figure(figsize=(12, 6))
    plt.plot(train_data.index, train_data['sales'], label='Dados de Treinamento')
    plt.plot(test_data.index, test_data['sales'], label='Vendas Reais (Teste)')
    plt.plot(forecast['date'], forecast['forecast_sales'], label='Previsão', linestyle='--')
    plt.legend()
    plt.title(f'Previsão vs. Vendas Reais - Loja {store} Item {item}')
    plt.xlabel('Data')
    plt.ylabel('Vendas')
    if save:
        plt.savefig(f'forecast_vs_actual_{store}_{item}.png')
    else:
        plt.show()
    
    # Plotar resíduos
    if 'actual_sales' in forecast.columns and 'forecast_sales' in forecast.columns:
        plt.figure(figsize=(10, 5))
        residuals = forecast['actual_sales'] - forecast['forecast_sales']
        plt.plot(forecast['date'], residuals, label='Resíduos')
        plt.hlines(0, forecast['date'].min(), forecast['date'].max(), colors='r', linestyles='dashed')
        plt.legend()
        plt.title(f'Residuals - Loja {store} Item {item}')
        plt.xlabel('Data')
        plt.ylabel('Resíduo')
        if save:
            plt.savefig(f'residuals_{store}_{item}.png')
        else:
            plt.show()

# Função para prever usando o modelo de duas etapas
def two_step_forecast_pipeline(train_data: pd.DataFrame, test_data: pd.DataFrame, features: list):
    # Primeira Etapa: Classificação
    train_data['is_sale'] = train_data['sales'] > 0
    test_data['is_sale'] = test_data['sales'] > 0
    
    X_train = train_data[features]
    y_train = train_data['is_sale']
    X_test = test_data[features]
    y_test = test_data['is_sale']
    
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)
    y_pred_class = clf.predict(X_test)
    
    logging.info("Relatório de Classificação:")
    logging.info(classification_report(y_test, y_pred_class))
    
    # Segunda Etapa: Regressão
    X_train_reg = X_train[train_data['is_sale'] == True]
    y_train_reg = train_data[train_data['is_sale'] == True]['sales']
    X_test_reg = X_test[test_data['is_sale'] == True]
    y_test_reg = test_data[test_data['is_sale'] == True]['sales']
    
    reg = RandomForestRegressor(n_estimators=100, random_state=42)
    reg.fit(X_train_reg, y_train_reg)
    y_pred_reg = reg.predict(X_test_reg)
    
    # Reunir previsões finais
    final_predictions = np.zeros(len(test_data))
    final_predictions[test_data['is_sale'] == True] = y_pred_reg
    
    # Calcular métricas
    rmse = mean_squared_error(test_data['sales'], final_predictions, squared=False)
    mae = mean_absolute_error(test_data['sales'], final_predictions)
    mask = test_data['sales'] != 0
    mape = mean_absolute_percentage_error(test_data['sales'][mask], final_predictions[mask]) * 100
    
    logging.info(f"RMSE: {rmse}, MAE: {mae}, MAPE: {mape}%")
    return rmse, mae, mape

# Pipeline completo para previsão de estoque dos itens por loja
def run_forecast_pipeline_for_top_items_per_store(query: str, forecast_table_full_id: str, model_type='Prophet', test_period_days=30, top_n=3, save_csv=False, csv_path='forecast_results.csv'):
    client = initialize_bigquery_client()
    raw_data = get_historical_data_from_bigquery(query, client)

    # Verificar se as colunas necessárias existem
    required_columns = {'store', 'item', 'date', 'sales'}
    if not required_columns.issubset(raw_data.columns):
        missing = required_columns - set(raw_data.columns)
        raise ValueError(f"As colunas a seguir estão faltando nos dados: {missing}")

    # Obter lista de lojas únicas
    stores = raw_data['store'].unique()
    all_forecasts = []

    for store in stores:
        logging.info(f"\nProcessando loja {store}...")
        store_data = raw_data[raw_data['store'] == store]
        total_sales_per_item = store_data.groupby('item')['sales'].sum().reset_index()
        top_items = total_sales_per_item.sort_values(by='sales', ascending=False).head(top_n)['item']  # Seleciona top N itens

        for item in top_items:
            logging.info(f"Processando item {item} na loja {store}...")
            item_data = store_data[store_data['item'] == item]
            prepared_data = prepare_sales_data(item_data, apply_log=False, aggregate_weekly=False)  # Mantém dados diários

            # Filtrar itens com muitas vendas zero
            if not filter_zero_sales(prepared_data, threshold=0.9):
                logging.warning(f"Item {item} na loja {store} possui alta proporção de vendas zero. Aplicando modelo de duas etapas.")
                
                # Engenharia de features para modelos de duas etapas
                prepared_data['day_of_week'] = prepared_data.index.dayofweek
                prepared_data['month'] = prepared_data.index.month
                # Supondo que 'event' seja uma coluna binária indicando eventos promocionais
                # Preencher valores ausentes com 0
                event_data = raw_data[(raw_data['store'] == store) & (raw_data['item'] == item)][['date', 'event']]
                event_data = event_data.set_index('date').reindex(prepared_data.index, fill_value=0)
                prepared_data['event'] = event_data['event']

                features = ['day_of_week', 'month', 'event']
                
                # Dividir em treino e teste
                cutoff_date = prepared_data.index.max() - pd.Timedelta(days=test_period_days)
                train_data = prepared_data[prepared_data.index <= cutoff_date]
                test_data = prepared_data[prepared_data.index > cutoff_date]

                if len(train_data) < 2 or len(test_data) < 1:
                    logging.warning(f"Dados insuficientes para treinamento ou teste para item {item} na loja {store}. Pulando...")
                    continue

                try:
                    # Aplicar o modelo de duas etapas
                    rmse, mae, mape = two_step_forecast_pipeline(train_data, test_data, features)
                    
                    logging.info(f"Loja {store}, Item {item} - RMSE: {rmse:.2f}, MAE: {mae:.2f}, MAPE: {mape:.2f}%")
                    
                    # Preparar DataFrame de previsão
                    forecast_df = pd.DataFrame({
                        'date': test_data.index,
                        'forecast_sales': np.where(test_data['is_sale'], test_data['sales'], 0),  # Substituir com previsões reais do modelo de duas etapas
                        'actual_sales': test_data['sales'].values,
                        'store': store,
                        'item': item,
                        'rmse': rmse,
                        'mae': mae,
                        'mape': mape,
                        'model': 'Two-Step'
                    })
                    
                    all_forecasts.append(forecast_df)
                    
                    # Plotar previsões vs. vendas reais e resíduos
                    plot_forecast_with_actual(train_data, test_data, forecast_df, store, item)
                    
                except Exception as e:
                    logging.error(f"Erro ao aplicar modelo de duas etapas para item {item} na loja {store}: {e}")
                    continue

                continue  # Pular para o próximo item após aplicar o modelo de duas etapas

            # Continuar com o fluxo normal de modelagem para itens com menos de 90% de vendas zero
            # Detectar outliers
            outliers = detect_outliers(prepared_data, threshold=1.5)
            if not outliers.empty:
                logging.warning(f"Detectados {len(outliers)} outliers no item {item} na loja {store}. Removendo outliers.")
                lower_bound = prepared_data['sales'].quantile(0.25) - 1.5 * (prepared_data['sales'].quantile(0.75) - prepared_data['sales'].quantile(0.25))
                upper_bound = prepared_data['sales'].quantile(0.75) + 1.5 * (prepared_data['sales'].quantile(0.75) - prepared_data['sales'].quantile(0.25))
                prepared_data = prepared_data[(prepared_data['sales'] >= lower_bound) & (prepared_data['sales'] <= upper_bound)]

            # Reafirmar frequência e preencher lacunas
            prepared_data = prepared_data.asfreq('D').fillna(0)

            # Verificar frequência
            freq = pd.infer_freq(prepared_data.index)
            logging.debug(f"Frequência dos dados após limpeza: {freq}")
            if freq != 'D':
                logging.warning(f"Frequência inesperada: {freq}. Corrigindo para diária.")
                prepared_data = prepared_data.asfreq('D').fillna(0)

            # Verificar se há datas faltantes após remoção de outliers
            missing_dates = prepared_data.asfreq('D').index.difference(prepared_data.index)
            if len(missing_dates) > 0:
                logging.warning(f"Datas faltantes após limpeza: {len(missing_dates)}. Preenchendo com zero.")
                prepared_data = prepared_data.asfreq('D').fillna(0)

            # Visualizar a série temporal
            visualize_series(prepared_data, store, item)

            if len(prepared_data) < (test_period_days + 1):
                logging.warning(f"Dados insuficientes após limpeza para item {item} na loja {store}. Necessário pelo menos {test_period_days + 1} dias, encontrado {len(prepared_data)} dias. Pulando...")
                continue

            try:
                # Definir a data de corte dinâmica: últimos 'test_period_days' dias para teste
                cutoff_date = prepared_data.index.max() - pd.Timedelta(days=test_period_days)
                train_data = prepared_data[prepared_data.index <= cutoff_date]
                test_data = prepared_data[prepared_data.index > cutoff_date]

                # Verificação de tamanhos
                steps = len(test_data)
                logging.debug(f"Treino: {len(train_data)} dias, Teste: {steps} dias")

                if len(train_data) < 2 or steps < 1:
                    logging.warning(f"Dados insuficientes para treinamento ou teste para item {item} na loja {store}, pulando...")
                    continue

                if model_type == 'ARIMA':
                    # Ajustar o modelo ARIMA usando sales (sem transformação)
                    model = fit_auto_arima_model(train_data['sales'])

                    # Validação cruzada
                    rmse_cv = cross_validate_arima(train_data['sales'], fit_auto_arima_model, splits=5)

                    if model is None:
                        logging.error(f"Não foi possível ajustar o modelo ARIMA para item {item} na loja {store}.")
                        continue

                    # Prever para o período de teste
                    forecast = model.predict(n_periods=steps)

                    # Se a série foi diferenciada, acumular as previsões
                    if model.order[1] > 0:
                        forecast = np.cumsum(forecast) + train_data['sales'].iloc[-1]

                    # Limitar previsões a valores não negativos
                    forecast_values = np.maximum(forecast, 0)

                elif model_type == 'Prophet':
                    # Ajustar o modelo Prophet usando sales
                    model = fit_prophet_model(train_data['sales'])

                    # Realizar validação cruzada com Prophet (opcional)
                    # df_cv = cross_validation(model, initial='365 days', period='30 days', horizon='30 days', parallel="processes")
                    # df_p = performance_metrics(df_cv)
                    # logging.info(f"Métricas de validação cruzada Prophet:\n{df_p}")

                    # Prever para o período de teste
                    forecast_values = predict_prophet_model(model, steps)

                    # Limitar previsões a valores não negativos
                    forecast_values = np.maximum(forecast_values, 0)

                else:
                    logging.error(f"Tipo de modelo '{model_type}' não reconhecido.")
                    continue

                actual_values = test_data['sales'].values

                # Verificação de tamanhos após previsão
                logging.debug(f"Tamanho de forecast_values: {len(forecast_values)}")
                logging.debug(f"Tamanho de actual_values: {len(actual_values)}")
                if len(forecast_values) != len(actual_values):
                    raise ValueError(f"Tamanho da previsão ({len(forecast_values)}) não corresponde ao tamanho dos valores reais ({len(actual_values)}).")

                # Calcular métricas de avaliação
                rmse, mae, mape = calculate_metrics(actual_values, forecast_values)

                logging.info(f"Loja {store}, Item {item} - RMSE: {rmse:.2f}, MAE: {mae:.2f}, MAPE: {mape:.2f}%")

                # Preparar DataFrame de previsão
                forecast_df = pd.DataFrame({
                    'date': test_data.index,
                    'forecast_sales': forecast_values,
                    'actual_sales': actual_values,
                    'store': store,
                    'item': item,
                    'rmse': rmse,
                    'mae': mae,
                    'mape': mape,
                    'rmse_cv': rmse_cv if model_type == 'ARIMA' else np.nan,
                    'order': str(model.order) if model_type == 'ARIMA' else 'Prophet'
                })

                all_forecasts.append(forecast_df)

                # Plotar previsões vs. vendas reais e resíduos
                plot_forecast_with_actual(train_data, test_data, forecast_df, store, item)

            except Exception as e:
                logging.error(f"Erro ao processar item {item} na loja {store}: {e}")
                continue

    if all_forecasts:
        final_forecast = pd.concat(all_forecasts, ignore_index=True)
        
        # Armazenar previsões na tabela de previsões no BigQuery
        store_forecast_results(final_forecast, forecast_table_full_id, client)
        
        # Salvar previsões em um arquivo CSV localmente
        if save_csv:
            final_forecast.to_csv(csv_path, index=False)
            logging.info(f"Previsões salvas com sucesso no arquivo {csv_path}.")
    else:
        logging.warning("Nenhuma previsão foi gerada.")

# Definir a consulta SQL para carregar os dados
query = f"""
SELECT store, item, date, sales
FROM `{full_data_table_id}`
"""

# Executar o pipeline, especificando a tabela de previsões e o tipo de modelo ('ARIMA' ou 'Prophet')
run_forecast_pipeline_for_top_items_per_store(
    query,
    f'{project_id}.{dataset_id}.{forecast_table_id}',
    model_type='Prophet',   # Alterar para 'ARIMA' se preferir usar ARIMA
    test_period_days=28,    # Definir o período de teste para os últimos 28 dias
    top_n=3,                # Número de itens a serem previstos por loja
    save_csv=True,          # Habilita o salvamento em CSV
    csv_path='previsoes_ds_market.csv'  # Caminho e nome do arquivo CSV
)
